# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 1936, done.
remote: Counting objects: 100% (318/318), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 1936 (delta 190), reused 307 (delta 180), pack-reused 1618 (from 1)
Receiving objects: 100% (1936/1936), 128.15 MiB | 22.94 MiB/s, done.
Resolving deltas: 100% (1135/1135), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 30.0 MB/s eta 0:00:00


In [4]:
! cd /kaggle/working/RECsys_Challenge2024 && python run_compile_all_cython.py

run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1252:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function '__pyx_pf_32MatrixFactorization

In [5]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os

np.random.seed(42)

## Import the repository

In [6]:
from github import Github, Auth

# Authenticate using a personal access token
auth_token = Auth.Token(token)
github_client = Github(auth=auth_token)

# Define the repository name you want to find
target_repo_name = 'RECsys_Challenge2024'
repo = None

# Search for the repository in the user's repositories
try:
    for repository in github_client.get_user().get_repos():
        if repository.name == target_repo_name:
            repo = repository
            print(f"Repository '{target_repo_name}' found.")
            break
    if repo is None:
        print(f"Repository '{target_repo_name}' not found.")
except Exception as e:
    print("An error occurred while accessing the repositories:", e)

Repository 'RECsys_Challenge2024' found.


In [7]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    """
    Uploads a file from Kaggle to GitHub, updating it if it already exists in the repository,
    or creating it if it does not.

    Parameters:
    - filepath_kaggle: Path to the file in the Kaggle environment.
    - filepath_github: Target path in the GitHub repository where the file should be uploaded.
    - commit_message: Message for the commit on GitHub.
    """
    try:
        
        # Check if the file already exists in the GitHub repository
        contents = repo.get_contents(filepath_github)
        
        # If it exists, update the file
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(
                contents.path, commit_message, file.read(), contents.sha
            )
        print(f"File '{filepath_github}' updated successfully.")
    
    except Exception as e:
        
        # If the file does not exist, create it
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(
                filepath_github, commit_message, file.read()
            )
        print(f"File '{filepath_github}' created successfully.")

In [8]:
config = {
    'model': 'SLIMElasticNet',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_SLIMElasticNet.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [9]:
try:
    shutil.copyfile(
        f'/kaggle/working/RECsys_Challenge2024/TrainedModels/' \
        f'{config["model"]}Recommender/history_{config["model"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [10]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1764607 stored elements and shape (35736, 38121)>

In [11]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (38121, 94331)>

# Training

In [12]:
%cd /kaggle/working/RECsys_Challenge2024/

/kaggle/working/RECsys_Challenge2024


In [13]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [14]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

def objective_function_SLIMElasticNet(optuna_trial):
    
    recommender_instance = SLIMElasticNetRecommender(URM_train)
    
    full_hyperp = {
                   "topK": optuna_trial.suggest_int("topK", 0, 1500),
                   "l1_ratio": optuna_trial.suggest_float("l1_ratio", 0.01, 1.0, log=True),
                    "alpha": optuna_trial.suggest_float("alpha", 1e-4, 1e-1, log=True),
                    "positive_only": optuna_trial.suggest_categorical("positive_only", [True, False]),
                  }        
    
    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [15]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_SLIMElasticNet, n_trials=5)

[I 2024-11-18 15:29:16,869] Using an existing study with name 'hyperparameters_tuning_SLIMElasticNet' instead of creating a new one.


SLIMElasticNetRecommender: Processed 5285 (13.9%) in 5.00 min. Items per second: 17.61
SLIMElasticNetRecommender: Processed 10476 (27.5%) in 10.00 min. Items per second: 17.45
SLIMElasticNetRecommender: Processed 16115 (42.3%) in 15.00 min. Items per second: 17.90
SLIMElasticNetRecommender: Processed 22076 (57.9%) in 20.00 min. Items per second: 18.39
SLIMElasticNetRecommender: Processed 27757 (72.8%) in 25.00 min. Items per second: 18.50
SLIMElasticNetRecommender: Processed 34364 (90.1%) in 30.00 min. Items per second: 19.09
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 33.34 min. Items per second: 19.06
EvaluatorHoldout: Processed 35595 (100.0%) in 31.50 sec. Users per second: 1130


[I 2024-11-18 16:03:08,963] Trial 29 finished with value: 0.05560065284717418 and parameters: {'topK': 1463, 'l1_ratio': 0.9155267633233201, 'alpha': 0.00016009522332561155, 'positive_only': True}. Best is trial 16 with value: 0.060174874079423415.


SLIMElasticNetRecommender: Processed 9881 (25.9%) in 5.00 min. Items per second: 32.93
SLIMElasticNetRecommender: Processed 20760 (54.5%) in 10.00 min. Items per second: 34.60
SLIMElasticNetRecommender: Processed 31205 (81.9%) in 15.00 min. Items per second: 34.67
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 18.13 min. Items per second: 35.04
EvaluatorHoldout: Processed 35595 (100.0%) in 29.14 sec. Users per second: 1221


[I 2024-11-18 16:21:46,171] Trial 30 finished with value: 0.04507277529169484 and parameters: {'topK': 1127, 'l1_ratio': 0.20903648278331885, 'alpha': 0.001290671918712393, 'positive_only': True}. Best is trial 16 with value: 0.060174874079423415.


...

[I 2024-11-18 17:22:06,173] Trial 31 finished with value: 0.05979242559035874 and parameters: {'topK': 981, 'l1_ratio': 0.5259982598596122, 'alpha': 0.0001030944717322155, 'positive_only': True}. Best is trial 16 with value: 0.060174874079423415.


SLIMElasticNetRecommender: Processed 3601 ( 9.4%) in 5.00 min. Items per second: 12.00
SLIMElasticNetRecommender: Processed 7221 (18.9%) in 10.00 min. Items per second: 12.03
SLIMElasticNetRecommender: Processed 10811 (28.4%) in 15.00 min. Items per second: 12.01
SLIMElasticNetRecommender: Processed 14459 (37.9%) in 20.00 min. Items per second: 12.05
SLIMElasticNetRecommender: Processed 18187 (47.7%) in 25.00 min. Items per second: 12.12
SLIMElasticNetRecommender: Processed 22143 (58.1%) in 30.00 min. Items per second: 12.30
SLIMElasticNetRecommender: Processed 26100 (68.5%) in 35.00 min. Items per second: 12.43
SLIMElasticNetRecommender: Processed 29960 (78.6%) in 40.01 min. Items per second: 12.48
SLIMElasticNetRecommender: Processed 34000 (89.2%) in 45.01 min. Items per second: 12.59
SLIMElasticNetRecommender: Processed 37829 (99.2%) in 50.01 min. Items per second: 12.61
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 50.44 min. Items per second: 12.60
EvaluatorHoldout: Proce

[I 2024-11-18 18:13:08,838] Trial 32 finished with value: 0.059590570282519476 and parameters: {'topK': 930, 'l1_ratio': 0.2634221889328485, 'alpha': 0.00023964506034263012, 'positive_only': True}. Best is trial 16 with value: 0.060174874079423415.


SLIMElasticNetRecommender: Processed 2979 ( 7.8%) in 5.00 min. Items per second: 9.92
SLIMElasticNetRecommender: Processed 5973 (15.7%) in 10.00 min. Items per second: 9.95
SLIMElasticNetRecommender: Processed 8929 (23.4%) in 15.00 min. Items per second: 9.92
SLIMElasticNetRecommender: Processed 11821 (31.0%) in 20.00 min. Items per second: 9.85
SLIMElasticNetRecommender: Processed 14759 (38.7%) in 25.00 min. Items per second: 9.84
SLIMElasticNetRecommender: Processed 17700 (46.4%) in 30.00 min. Items per second: 9.83
SLIMElasticNetRecommender: Processed 20749 (54.4%) in 35.00 min. Items per second: 9.88
SLIMElasticNetRecommender: Processed 23787 (62.4%) in 40.01 min. Items per second: 9.91
SLIMElasticNetRecommender: Processed 26901 (70.6%) in 45.01 min. Items per second: 9.96
SLIMElasticNetRecommender: Processed 29998 (78.7%) in 50.01 min. Items per second: 10.00
SLIMElasticNetRecommender: Processed 33155 (87.0%) in 55.01 min. Items per second: 10.05
SLIMElasticNetRecommender: Process

[I 2024-11-18 19:16:51,628] Trial 33 finished with value: 0.059856563143118637 and parameters: {'topK': 733, 'l1_ratio': 0.11920660306420122, 'alpha': 0.00013665010229927006, 'positive_only': True}. Best is trial 16 with value: 0.060174874079423415.


## Some optuna visualizations on recommender parameters

In [16]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [17]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [18]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'/kaggle/working/TrainedModels/{config["model"]}Recommender/best_params_{config["model"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = SLIMElasticNetRecommender(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

SLIMElasticNetRecommender: Processed 2552 ( 6.7%) in 5.00 min. Items per second: 8.50
SLIMElasticNetRecommender: Processed 5129 (13.5%) in 10.00 min. Items per second: 8.54
SLIMElasticNetRecommender: Processed 7710 (20.2%) in 15.00 min. Items per second: 8.56
SLIMElasticNetRecommender: Processed 10294 (27.0%) in 20.00 min. Items per second: 8.58
SLIMElasticNetRecommender: Processed 12856 (33.7%) in 25.00 min. Items per second: 8.57
SLIMElasticNetRecommender: Processed 15460 (40.6%) in 30.01 min. Items per second: 8.59
SLIMElasticNetRecommender: Processed 18024 (47.3%) in 35.01 min. Items per second: 8.58
SLIMElasticNetRecommender: Processed 20653 (54.2%) in 40.01 min. Items per second: 8.60
SLIMElasticNetRecommender: Processed 23239 (61.0%) in 45.01 min. Items per second: 8.61
SLIMElasticNetRecommender: Processed 25849 (67.8%) in 50.01 min. Items per second: 8.61
SLIMElasticNetRecommender: Processed 28453 (74.6%) in 55.01 min. Items per second: 8.62
SLIMElasticNetRecommender: Processed

# Testing

Create the recommendations for the submission. 

In [19]:
def create_submission(data_target_users_test, recommender_instance, cutoff=10, output_file=f"/kaggle/working/submission_{config['model']}.csv"):
    
    target_result = []

    for target in data_target_users_test["user_id"]:
        target_result.append(recommender_instance.recommend(target, cutoff=cutoff, remove_seen_flag=True))

    user_ids = data_target_users_test["user_id"]
    formatted_data = {
        "user_id": user_ids,
        "item_list": [" ".join(map(str, items)) for items in target_result]
    }

    submission_df = pd.DataFrame(formatted_data)
    submission_df.to_csv(output_file, index=False, header=["user_id", "item_list"])

    print(f"Submission file saved as {output_file}")

In [20]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance)

Submission file saved as /kaggle/working/submission_SLIMElasticNet.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [21]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}.json', 
            f'TrainedModels/{config["model"]}Recommender/best_params_{config["model"]}.json', 
            f'{config["model"]} tuning results (from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'/kaggle/working/RECsys_Challenge2024/TrainedModels/{config["model"]}Recommender/'\
        f'best_params_{config["model"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}.json'
    )

File 'TrainedModels/SLIMElasticNetRecommender/best_params_SLIMElasticNet.json' updated successfully.


Save the history of the tuned model.

In [22]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'TrainedModels/{config["model"]}Recommender/history_{config["model"]}.db',
        f'Tuning {config["model"]} db updated results (from kaggle notebook)'
    )

File 'TrainedModels/SLIMElasticNetRecommender/history_SLIMElasticNet.db' updated successfully.


Save the best trained model and its submission.

In [ ]:
if config['save_github'] and config['tune_best_params']:
    
    upload_file(
                f'/kaggle/working/submission_{config["model"]}.csv', 
                f'TrainedModels/{config["model"]}Recommender/Submission/'\
                    f'submission_{config["model"]}.csv', 
                f'New {config["model"]} submission (from kaggle notebook)'
            )
    
    recommender_instance.save_model(folder_path='/kaggle/working/', file_name=f'best_{config["model"]}_tuned')

    try:
        upload_file(
            f'/kaggle/working/best_{config["model"]}_tuned.zip',  
            f'TrainedModels/{config["model"]}Recommender/best_{config["model"]}_tuned.zip', 
            f'New {config["model"]} recommender tuned with best parameters (from kaggle notebook)'
        )
    except Exception as e:
        print(f"\nThe best recommender was not saved on GitHub because its size is more than 50 MB.")

File 'TrainedModels/SLIMElasticNetRecommender/Submission/submission_SLIMElasticNet.csv' updated successfully.
SLIMElasticNetRecommender: Saving model in file '/kaggle/working/best_SLIMElasticNet_tuned'
SLIMElasticNetRecommender: Saving complete

The best recommender was not saved on GitHub because its size is more than 50 MB.
